Notebook features:
- parse through polygon api ticker(stock+crypto) data JSON files from google drive
- can initialise target features, and
- can initialise multiple sklearn models ( LinearRegression, XGBRegressor, ElasticNet)
- *not implemented: ensembles multiple models to do predictions get highest scoring (r2)*
- saves trained models, x_scaler, y_scaler as Pickle to s3.

Template2 formats dictionaries into Dataframes like this:
```
ticker|index|open |close|...|
-------------------------
 AAPL|    0|240.2|241.5|...|
     |    1|238.1|239.2|...|
     |    2|243.1|241.3|...|
 NVDA|    0|120.4|121.7|...|
     |    1|128.9|129.1|...|
     |    2|133.1|121.3|...|
```

# 1. Setup

## 1.1 Mount Google Drive

In [ ]:
# Mount to this notebook to access JSON saved there.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.2 Imports

In [ ]:
!pip install xgboost==2.1.0

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt

# Data Preprocessing
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler # https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#robustscaler

# Regression Models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
# from sklearn.metrics import r2_score
# from sklearn.metrics import mean_squared_error

# Evaluation
from sklearn.metrics import mean_absolute_error

# Prediction
from sklearn.base import clone

## 1.3 Constants

In [ ]:
# To hold dataframes created from extracted JSON files

#   a: don't think my google drive can be accessed by other users.
#   a: if cannot, download the .json files into your google drive and use your own source path.
SOURCE_DIRECTORY = f'/content/drive/MyDrive/Colab Notebooks/RoboTrader-Data-Random_Combined'

# Output related
NOTEBOOK_NAME = 'Template1'

# add more if required.

# 2. Data Preprocessing

In [ ]:
# The number of 10-min intervals are: 6/hour, 39/day, 195/week(5days), 819/month(21days), 2457/quarter(63days), 9828(84days).
WINDOW_DATAPOINTS_QUANTITY = 819 # Datapoints in total.
FUTURE_DATAPOINTS_QUANTITY = 39  # Datapoints for test.

## 2.1 Read JSON from Google Drive, use as Dataframe

todo: read from s3 instead from google drive

In [ ]:
# Data Preprocessing Variables - for ticker data extracted from polygon API.
UNWANTED_FEATURES = ['open', 'high', 'low', 'volume', 'otc', 'timestamp', 'transactions']

In [ ]:
def json_to_dataframes(source_directory):
    all_data = []

    # Iterate over all files in the source directory
    for filename in os.listdir(source_directory):

        filepath = os.path.join(source_directory, filename)
        # Check if the file is not empty
        if os.path.getsize(filepath) <= 0:
            print(f"Skipping empty file {filename}")
            continue

        try:
            # Open and read the json file
            with open(filepath, 'r') as file:
                data_dict = json.load(file)

            for ticker in data_dict:
                # Create a DataFrame from the JSON data
                current_df = pd.DataFrame(data=data_dict[ticker]['data'],
                                                columns=data_dict[ticker]['columns'],
                                                index=data_dict[ticker]['index'])
                # # Do not add to dictionary if df is empty
                # if (len(current_df.columns) <= 0):
                #     continue
                current_df.columns = current_df.columns.str.replace(' ', '')
                if current_df.shape[0] <= WINDOW_DATAPOINTS_QUANTITY:
                    print(f'Excluded {ticker}. Has {current_df.shape[0]} samples when {WINDOW_DATAPOINTS_QUANTITY} is required.')
                    continue
                current_df['ticker'] = ticker  # Add ticker column for MultiIndex
                current_df.drop(labels=UNWANTED_FEATURES, axis=1, inplace=True)
                all_data.append(current_df[-(WINDOW_DATAPOINTS_QUANTITY+FUTURE_DATAPOINTS_QUANTITY):])

        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error processing file {filename}: {e}")

        # Concatenate all DataFrames into a single MultiIndex DataFrame
        if all_data:
            combined_df = pd.concat(all_data)
            combined_df.reset_index(inplace=True)
            combined_df.set_index(['ticker', 'index'], inplace=True)
        else:
            combined_df = pd.DataFrame()  # Return an empty DataFrame if no data is collected

    return combined_df

In [ ]:
# Load JSON to global MultiIndex Dataframe
df_raw = json_to_dataframes(SOURCE_DIRECTORY)

Excluded FXBY. Has 208 samples when 819 is required.
Excluded WLTW. Has 0 samples when 819 is required.
Excluded VNORP. Has 51 samples when 819 is required.
Excluded PBSTV. Has 0 samples when 819 is required.
Excluded BMYMP. Has 132 samples when 819 is required.
Excluded XON. Has 0 samples when 819 is required.
Excluded AMCCF. Has 46 samples when 819 is required.
Excluded 0P0001I1JH. Has 0 samples when 819 is required.
Excluded UHID. Has 11 samples when 819 is required.
Excluded BOAPL. Has 0 samples when 819 is required.
Excluded OCLCF. Has 25 samples when 819 is required.
Excluded INPAP. Has 17 samples when 819 is required.
Excluded RSMDF. Has 53 samples when 819 is required.
Excluded TAP-A. Has 0 samples when 819 is required.
Excluded BRK-B. Has 0 samples when 819 is required.
Excluded CUKPF. Has 135 samples when 819 is required.
Excluded CNP-PB. Has 0 samples when 819 is required.
Excluded HBANN. Has 0 samples when 819 is required.
Excluded 0P0000KQL0. Has 0 samples when 819 is requ

In [ ]:
assert df_raw.shape[0] > 0
print(df_raw.shape)
print(df_raw.describe())
print(df_raw.tail())

(435006, 2)
               close           vwap
count  435006.000000  435006.000000
mean      332.463440     332.469914
std      2982.027959    2982.175936
min         0.050820       0.050700
25%        56.640000      56.641300
50%       109.610000     109.620000
75%       213.070000     213.077875
max     68429.380000   68400.020500
                   close    vwap
ticker    index                 
X:USDCUSD 104724  1.0000  1.0000
          104725  1.0000  1.0000
          104726  1.0000  1.0000
          104727  0.9999  0.9999
          104728  0.9999  0.9999


## 2.2 Feature Engineering

In [ ]:
df_cleaned = df_raw.copy()
print(df_cleaned.shape)
print(df_cleaned.tail())

(435006, 2)
                   close    vwap
ticker    index                 
X:USDCUSD 104724  1.0000  1.0000
          104725  1.0000  1.0000
          104726  1.0000  1.0000
          104727  0.9999  0.9999
          104728  0.9999  0.9999


### 2.2.1 Separate by S&P sectors

In [ ]:
# TODO.
# ticker_sectors = pd.read_csv('https://github.com/datasets/s-and-p-500-companies/raw/main/data/constituents.csv')

### 2.2.2 Lag Data as Features

In [ ]:
FEATURES = ['vwap'] # Volume Weighted Average Price (VWAP) is the average price of a ticker weighted by the total trading volume.
LABEL = ['close']

In [ ]:
def add_lagged_features(df_main, label, features, future_window):
    """
    Adds lagged features for machine learning models.

    Parameters:
    - df_main (DataFrame): The input MultiIndex DataFrame.
    - lag_cols (list): List of column names to use for creating lag features.
    - future_window (int): The number of lags (periods) to create.

    Returns:
    - DataFrame: The transformed DataFrame with added lag features.
    """
    all_lagged_data = []

    # gets pandas dataframe's Index object e.g., Index(['AAPL', 'NVDA', ...])
    tickers = df_main.index.get_level_values('ticker').unique()

    # list of columns used to create lags
    for ticker in tickers:
        # Select the data for the current ticker
        df = df_main.loc[ticker].copy()

        # Create lagged features for the past N periods
        for lag in range(1, future_window):
            df[f'lag_vwap_{lag}'] = df['vwap'].shift(lag)

        # Drop rows with NaN values created by the lagged features
        df.dropna(inplace=True)

        # Reintroduce the ticker column for concatenation later
        df['ticker'] = ticker
        all_lagged_data.append(df)

    # Use only lagging close prices as Feature columns
    for i in range(1, future_window):
        FEATURES.append(f'lag_vwap_{i}')

    # Concatenate all lagged DataFrames into a single MultiIndex DataFrame
    if all_lagged_data:
        combined_lagged_df = pd.concat(all_lagged_data)
        combined_lagged_df.reset_index(inplace=True)
        combined_lagged_df.set_index(['ticker', 'index'], inplace=True)
    else:
        combined_lagged_df = pd.DataFrame()  # Return an empty DataFrame if no data is collected

    return combined_lagged_df


### 2.2.3 Feature Selection

In [ ]:
# TODO: With Feature Selection, we will be able to introduce more features in above sections, before choosing the most significant ones.

### Execute

In [ ]:
df_feature_engineered = add_lagged_features(df_cleaned, LABEL, FEATURES, FUTURE_DATAPOINTS_QUANTITY)

In [ ]:
print(df_feature_engineered.shape)
print(df_feature_engineered.tail())

(415740, 40)
                   close    vwap  lag_vwap_1  lag_vwap_2  lag_vwap_3  \
ticker    index                                                        
X:USDCUSD 104724  1.0000  1.0000      1.0000      0.9999      1.0000   
          104725  1.0000  1.0000      1.0000      1.0000      0.9999   
          104726  1.0000  1.0000      1.0000      1.0000      1.0000   
          104727  0.9999  0.9999      1.0000      1.0000      1.0000   
          104728  0.9999  0.9999      0.9999      1.0000      1.0000   

                  lag_vwap_4  lag_vwap_5  lag_vwap_6  lag_vwap_7  lag_vwap_8  \
ticker    index                                                                
X:USDCUSD 104724      1.0000      1.0000      1.0000         1.0         1.0   
          104725      1.0000      1.0000      1.0000         1.0         1.0   
          104726      0.9999      1.0000      1.0000         1.0         1.0   
          104727      1.0000      0.9999      1.0000         1.0         1.0   
  

## 2.3 Train-Test Split and Scale

todo: use RobustScaler
https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#robustscaler

In [ ]:
# Split dataset
def train_test_split_Scale(df, features, label, future_datapoints):
    """
    Splits the DataFrame into train and test sets and scales the features.

    Parameters:
    - df (DataFrame): The input MultiIndex DataFrame with all features and labels.
    - features (list): List of feature columns to use for scaling.
    - label (str): The label column to predict.
    - future_datapoints (int): Number of datapoints to include in the test set from the end of each series.

    Returns:
    - X_train (DataFrame): Scaled training features.
    - X_test (DataFrame): Scaled testing features.
    - y_train (Series): Training labels.
    - y_test (Series): Testing labels.
    """
    train_data = []
    test_data = []

    tickers = df.index.get_level_values('ticker').unique()

    for ticker in tickers:
        ticker_df = df.loc[ticker]
        train_df = ticker_df[:-future_datapoints].copy()
        test_df = ticker_df[-future_datapoints:].copy()

        # Reintroduce the ticker column for concatenation later
        train_df['ticker'] = ticker
        test_df['ticker'] = ticker

        train_data.append(train_df)
        test_data.append(test_df)

    # Concatenate train and test data into separate MultiIndex DataFrames
    train_data = pd.concat(train_data)
    test_data = pd.concat(test_data)

    # Reset and set index to handle concatenation properly
    train_data.reset_index(inplace=True, drop=False)
    train_data.set_index(['ticker', 'index'], inplace=True)
    test_data.reset_index(inplace=True, drop=False)
    test_data.set_index(['ticker', 'index'], inplace=True)

    # Split into features and labels
    X_train = train_data[features].astype(float)
    y_train = train_data[label].astype(float)
    X_test = test_data[features].astype(float)
    y_test = test_data[label].astype(float)

    for ticker in tickers:
        row_labels = pd.IndexSlice[ticker, :]

        # Apply RobustScaler to the features. Fitted scalers are stored for later sections.
        dictionary_X_train_scaler[ticker] = RobustScaler().fit(X_train.loc[row_labels])
        dictionary_X_test_scaler[ticker] = RobustScaler().fit(X_test.loc[row_labels])
        dictionary_y_train_scaler[ticker] = RobustScaler().fit(y_train.loc[row_labels])
        dictionary_y_test_scaler[ticker] = RobustScaler().fit(y_test.loc[row_labels])

        X_train.loc[row_labels] = dictionary_X_train_scaler[ticker].transform(X_train.loc[row_labels])
        X_test.loc[row_labels] = dictionary_X_test_scaler[ticker].transform(X_test.loc[row_labels])
        y_train.loc[row_labels] = dictionary_y_train_scaler[ticker].transform(y_train.loc[row_labels])
        y_test.loc[row_labels] = dictionary_y_test_scaler[ticker].transform(y_test.loc[row_labels])

    return X_train, X_test, y_train, y_test

In [ ]:
# Hold fitted scalers to inverse scaling after predictions later
dictionary_X_train_scaler = {}
dictionary_X_test_scaler = {}
dictionary_y_train_scaler = {}
dictionary_y_test_scaler = {}

In [ ]:
# Train-Test Split
# train_test_split_Scale(df_dictionary_feature_engineered, FEATURES, LABEL, FUTURE_DATAPOINTS)
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split_Scale(df_feature_engineered, FEATURES, LABEL, FUTURE_DATAPOINTS_QUANTITY)

In [ ]:
# To visualise
print(df_X_train.shape, df_X_train.tail())
print(df_X_test.shape, df_X_test.tail())
print(df_y_train.shape, df_y_train.tail())
print(df_y_test.shape, df_y_test.tail())

(395967, 39)                   vwap  lag_vwap_1  lag_vwap_2  lag_vwap_3  lag_vwap_4  \
ticker    index                                                          
X:USDCUSD 104685   0.0         0.0        -1.0        -3.0        -1.0   
          104686   0.0         0.0         0.0        -1.0        -3.0   
          104687   1.0         0.0         0.0         0.0        -1.0   
          104688   1.0         1.0         0.0         0.0         0.0   
          104689   0.0         1.0         1.0         0.0         0.0   

                  lag_vwap_5  lag_vwap_6  lag_vwap_7  lag_vwap_8  lag_vwap_9  \
ticker    index                                                                
X:USDCUSD 104685         0.0         1.0         0.0       -35.0        -2.0   
          104686        -1.0         0.0         1.0         0.0       -35.0   
          104687        -3.0        -1.0         0.0         1.0         0.0   
          104688        -1.0        -3.0        -1.0         0.0    

# 3. Modular Sklearn Models

Using sklearn's Models' .predict method in `Train and Evaluate` and `Predict` sections later.

Thus, the models used here should be available in Sklearn.

## 3.1 Models

In [ ]:
# To hold multiple Keras Model objects across notebook so they can be iterated:
models = [] # new models
models_names = []
trained_models = {} # key: ticker, value: trained Model object

In [ ]:
# XGBRegressor parameters
n_estimators = 1000            # Number of boosted trees to fit. default = 100
max_depth = 5                  # Maximum tree depth for base learners. default = 3
learning_rate = 0.1            # Boosting learning rate (xgb's "eta"). default = 0.1
min_child_weight = 1           # Minimum sum of instance weight(hessian) needed in a child. default = 1
subsample = 1                  # Subsample ratio of the training instance. default = 1
colsample_bytree = 1           # Subsample ratio of columns when constructing each tree. default = 1
colsample_bylevel = 1          # Subsample ratio of columns for each split, in each level. default = 1
gamma = 0                      # Minimum loss reduction required to make a further partition on a leaf node of the tree. default=0

model_seed = 100

In [ ]:
# Set and append to models[]
# models = [LinearRegression(), Model2(), Model3(), etc.]
models = [
    LinearRegression(),
    ElasticNet(alpha=0.2, l1_ratio=0.2),
    # MLPRegressor(hidden_layer_sizes=(50, 25), max_iter=1000), # may not use. takes long time to train & performs worse than others.
    XGBRegressor(objective ='reg:squarederror',
                     seed=model_seed,
                     n_estimators=n_estimators,
                     max_depth=max_depth,
                     learning_rate=learning_rate,
                     min_child_weight=min_child_weight,
                     subsample=subsample,
                     colsample_bytree=colsample_bytree,
                     colsample_bylevel=colsample_bylevel,
                     gamma=gamma)
]

In [ ]:
for i, model in enumerate(models):
    model_name = str(model).split("(")[0]
    if model_name not in models_names:
        models_names.append(model_name)

In [ ]:
print(models_names)

['LinearRegression', 'ElasticNet', 'XGBRegressor']


# 4. Train, Evaluate, Predict

In [ ]:
# Predictions
predictions_close_price_dictionary = {} # key: ticker, value: prediction ('close' price)

## 4.1 Functions

### 4.1.1 Plot

In [ ]:
def plot_predictions(ticker, model_name, y_train, y_test, y_pred, mae, future_y_pred):
    fig, axs = plt.subplots(1, 1, layout='constrained')
    axs.plot(range(len(y_train)-2*FUTURE_DATAPOINTS_QUANTITY), y_train[FUTURE_DATAPOINTS_QUANTITY*2:], color='blue')
    axs.plot(range(len(y_train)-2*FUTURE_DATAPOINTS_QUANTITY,len(y_train)+len(y_test)-2*FUTURE_DATAPOINTS_QUANTITY), y_test, color='green')
    axs.plot(range(len(y_train)-2*FUTURE_DATAPOINTS_QUANTITY,len(y_train)+len(y_pred)-2*FUTURE_DATAPOINTS_QUANTITY), y_pred, color='red')
    plt.plot(range(len(y_train)+len(y_test)-2*FUTURE_DATAPOINTS_QUANTITY,len(y_train)+len(y_test)+len(future_y_pred)-2*FUTURE_DATAPOINTS_QUANTITY), future_y_pred, color='black')
    axs.set_title(f'{ticker}: {model_name} - MAE: {mae:.2f}')
    axs.set_xlabel('Period')
    axs.set_ylabel('$USD')

    plt.show()
    return

### 4.1.2 Train and Predict

In [ ]:
# Train, evaluate, and output a fitted model, R2 score, and rmse.
def individual_model_train_predict(ticker, model_name, model, X_train, X_test, y_train):
    model_fitted = model.fit(X = X_train[:-FUTURE_DATAPOINTS_QUANTITY],
                             y = y_train[:-FUTURE_DATAPOINTS_QUANTITY])
    y_pred = model_fitted.predict(X_train[-FUTURE_DATAPOINTS_QUANTITY:])
    return model_fitted, y_pred

### 4.1.3 Loss Function
todo: to explore using Mean absolute scaled error

In [ ]:
def calculate_mae(y_true, y_pred):
    """
    Compute mean absolute error (MAE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return mean_absolute_error(y_true, y_pred)

### 4.1.4 Ensemble

In [ ]:
# # Stacking (one form of ensemble learning)

# # imports
# import time
# from sklearn.metrics import PredictionErrorDisplay
# # from sklearn.model_selection import cross_val_predict, cross_validate

# # from mlxtend.regressor import StackingCVRegressor # d: not too sure what is the difference between this and the sklearn one
# from sklearn.ensemble import StackingRegressor

# # # create stack regressor
# # def create_stackingCV_regressor(models):
# #     return StackingCVRegressor(regressors=models,
# #                                     meta_regressor=xgboost,
# #                                     use_features_in_secondary=True) # ref: https://rasbt.github.io/mlxtend/user_guide/regressor/StackingCVRegressor/#api

# def create_stacking_regressor(models):
#     return StackingRegressor(
#         estimators=models,
#         final_estimator=xgboost) # ref: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html

# # afterwards we can compare the performance of each individual model vs the STACK
# # which is similar to the portion that you have done below, where you indicated " ### TODO: Shift this to ensemble section ###"

# # a: thanks, will look into it

In [ ]:
# # TODO
# def ensemble_models(models):
#     best_score = -9999999
#     best_model_fitted = None
#     y_pred = None

#     for model in models:

#         local_model = clone(model)
#         local_model_name = str(local_model).split("(")[0]
#         model_fitted, y_pred_output, score, rmse = individual_model_train_and_evaluate(ticker, local_model_name, local_model,
#                                                     X_train, X_test,
#                                                     y_train, y_test)
#         # print(f'{ticker}: {str(model_fitted).split("(")[0]}, score: {score}') # for debugging.
#         if score > best_score:
#             best_score = score
#             best_model_fitted = model_fitted
#             y_pred = y_pred_output

#     return best_model_fitted, best_score, rmse, y_pred

### 4.1.5 Prepare future datapoints for  predictions

In [ ]:
def data_to_supervised_learning(X_test, X_train, n_in, n_out=1, dropnan=True):
    X_train = X_train[n_in:]
    X_future = pd.concat([X_train, X_test], axis=0)
    return X_future.values[-FUTURE_DATAPOINTS_QUANTITY:]

## 4.2 Iterate through Functions

In [ ]:
# Iterate through models[], train and evaluate them.
def all_models_train_and_evaluate(models, df_X_train, df_X_test,
                       df_y_train, df_y_test):

    # Check dictionary lengths before continuing
    if ((df_X_train.shape[0] != df_y_train.shape[0])
        or (df_X_test.shape[0] != df_y_test.shape[0])
    ):
        raise Exception(f"Please make sure all dataframe lengths are equal.")

    tickers = df_X_train.index.get_level_values('ticker').unique()

    # Initialise variables
    for ticker in tickers:
        row_labels = pd.IndexSlice[ticker, :]
        local_X_train = df_X_train.loc[row_labels].values
        local_X_test = df_X_test.loc[row_labels].values
        local_y_train = df_y_train.loc[row_labels].values.ravel()
        local_y_test = df_y_test.loc[row_labels].values.ravel()

        lowest_mae = 9999999
        best_model_fitted = None
        ### TODO: Shift this to ensemble section ###
        for model in models:
            local_model = clone(model)
            local_model_name = str(local_model).split("(")[0]
            model_fitted, y_pred = individual_model_train_predict(ticker, local_model_name, local_model,
                                                        local_X_train, local_X_test, local_y_train)
            mae = calculate_mae(y_true=dictionary_y_test_scaler[ticker].inverse_transform(local_y_test.reshape(-1, 1)),
                                y_pred=dictionary_y_test_scaler[ticker].inverse_transform(y_pred.reshape(-1, 1)))
            # print(f'{ticker}: {str(model_fitted).split("(")[0]}, score: {score}') # for debugging.
            if mae < lowest_mae:
                lowest_mae = mae
                best_model_fitted = model_fitted

        # prepare future_x and predict future_y_pred
        future_X = data_to_supervised_learning(df_X_test.loc[row_labels], df_X_train.loc[row_labels], FUTURE_DATAPOINTS_QUANTITY, 1)
        print(future_X.shape)
        future_y_pred = best_model_fitted.predict(future_X)

        # inverse scaling
        df_dictionary_X_train_scale_inversed[ticker] = dictionary_X_train_scaler[ticker].inverse_transform(local_X_train)
        df_dictionary_X_test_scale_inversed[ticker] = dictionary_X_test_scaler[ticker].inverse_transform(local_X_test)
        df_dictionary_y_train_scale_inversed[ticker] = dictionary_y_train_scaler[ticker].inverse_transform(local_y_train.reshape(-1, 1))
        df_dictionary_y_test_scale_inversed[ticker] = dictionary_y_test_scaler[ticker].inverse_transform(local_y_test.reshape(-1, 1))
        y_pred = dictionary_y_test_scaler[ticker].inverse_transform(y_pred.reshape(-1, 1))
        future_X = dictionary_X_test_scaler[ticker].inverse_transform(future_X)
        future_y_pred = dictionary_y_test_scaler[ticker].inverse_transform(future_y_pred.reshape(-1, 1))

        best_model_fitted_name = str(best_model_fitted).split("(")[0]
        # if i < 2: # for sample visualisation
        print(f'''\nModel: {best_model_fitted_name}
        Ticker: {ticker}
        Mean Absolute Error: {mae}''')
        # uncomment to plot. takes a long time while running the notebook.
        plot_predictions(ticker, best_model_fitted_name, df_dictionary_y_train_scale_inversed[ticker], df_dictionary_y_test_scale_inversed[ticker], y_pred, mae, future_y_pred)

        # Store fitted model
        trained_models[ticker] = best_model_fitted
        # Store predictions
        predictions_close_price_dictionary[ticker] = np.round(future_y_pred.flatten(), 2)

        # # Store intercept
        # predictions_intercept_dictionary[ticker] = best_model_fitted.intercept_
        # # Store coefficient
        # if (best_model_fitted_name == "LinearRegression"):
        #     predictions_coef_dictionary[ticker] = best_model_fitted.coef_[0]
        # else:
        #     predictions_coef_dictionary[ticker] = best_model_fitted.coef_


In [ ]:
df_dictionary_X_train_scale_inversed = {}
df_dictionary_X_test_scale_inversed = {}
df_dictionary_y_train_scale_inversed = {}
df_dictionary_y_test_scale_inversed = {}

## 4.3 Execute

In [ ]:
all_models_train_and_evaluate(models, df_X_train, df_X_test,
                       df_y_train, df_y_test)

# 5. Prediction Post-processing

## 5.1 Functions

### 5.1.1 Save predictions (.json)

In [ ]:
from time import time
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

def save_dictionary_as_json(ticker, output_dictionary):
    output = output_dictionary[ticker]

    f=open(f'/content/drive/MyDrive/Colab Notebooks/RoboTrader-Predictions/{ticker}.json',"w")
    f.write(json.dumps(output,cls=NumpyEncoder))
    f.close()

    print(f'Saved {ticker}.json predictions')
    return


### 5.1.2 Save as local files (.pkl)

In [ ]:
import pickle
def save_as_local_file(ticker):
    model = trained_models[ticker]
    model_name = str(model).split("(")[0]

    if model_name in models_names:
        if model_name == 'XGBRegressor':
            # save trained XGBRegressor model
            with open(f'/content/drive/MyDrive/Colab Notebooks/RoboTrader-Models/model/{ticker}.pkl', 'wb') as f:
                pickle.dump(model.save_model, f)
        else:
            # save other trained models
            with open(f'/content/drive/MyDrive/Colab Notebooks/RoboTrader-Models/model/{ticker}.pkl', 'wb') as f:
                pickle.dump(model, f)  #serialize the object

        # save x_test_scaler
        with open(f'/content/drive/MyDrive/Colab Notebooks/RoboTrader-Models/x_scaler/{ticker}.pkl', 'wb') as f:
                pickle.dump(dictionary_X_test_scaler[ticker], f)  #serialize the object

        # save y_test_scaler
        with open(f'/content/drive/MyDrive/Colab Notebooks/RoboTrader-Models/y_scaler/{ticker}.pkl', 'wb') as f:
                pickle.dump(dictionary_y_test_scaler[ticker], f)  #serialize the object

    else:
        raise Exception(f"Please implement saving of {ticker}'s model: {model_name}.")

    print(f'Saved {ticker}.pkl model')
    return

### 5.1.3 Save scalers (.pkl)

In [ ]:
# dictionary_X_train_scaler = {}
# dictionary_X_test_scaler = {}
# dictionary_y_train_scaler = {}
# dictionary_y_test_scaler = {}

### 5.1.4 Upload to S3 Bucket

In [ ]:
# imports
!pip install boto3
import boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
def upload_to_s3(s3, ticker, prefix):
    models_bucket_name = "fourquant-robotrader-models"

    # Upload to S3 bucket
    with open(f'/content/drive/MyDrive/Colab Notebooks/RoboTrader-Models/{prefix}/{ticker}.pkl', "rb") as f:
        s3.upload_fileobj(f, models_bucket_name, f'{prefix}/{ticker}.pkl')

    print(f'Uploaded {prefix}/{ticker}.pkl')
    return

### 5.1.4 Iterate through all tickers

In [ ]:
# predictions_coef_dictionary[ticker] = [[-5.90832260e-01  5.43316834e-01  ...]]
def combine_predictions_to_dictionary():

    # Set up constants for uploading to S3 bucket
    with open(f'/content/drive/MyDrive/Colab Notebooks/env/aws_access_key.txt', 'r') as f:
        aws_access_key_id = f.read()

    with open(f'/content/drive/MyDrive/Colab Notebooks/env/aws_secret_key.txt', 'r') as f:
        aws_secret_access_key = f.read()

    # Set up S3 client
    s3 = boto3.client('s3',
                    aws_access_key_id = aws_access_key_id,
                    aws_secret_access_key = aws_secret_access_key)

    # Add ticker, predicted close prices to dictionary
    for i, ticker in enumerate(trained_models):
        output_dictionary = {}

        model_name = str(trained_models[ticker]).split("(")[0]
        index = list(range(len(predictions_close_price_dictionary[ticker]))), # generates ([0, 1, ... n],)

        # JSON output format
        if model_name == "LinearRegression" or model_name == "XGBRegressor" or model_name == "ElasticNet":
            output_dictionary[ticker] = {
                # "model": model_name,
                # "index": index,
                # "coef": predictions_coef_dictionary[ticker],
                # "intercept": predictions_intercept_dictionary[ticker]
                "tickerName": ticker,
                "predictions": predictions_close_price_dictionary[ticker]
            }

        else:
            raise Exception(f"Please ensure loop checks for {model_name} and outputs its JSON.")

        # save_dictionary_as_json(ticker, output_dictionary)
        save_as_local_file(ticker)
        upload_to_s3(s3, ticker, "model")
        upload_to_s3(s3, ticker, "x_scaler")
        upload_to_s3(s3, ticker, "y_scaler")


## 5.2 Execute

In [ ]:
combine_predictions_to_dictionary()

Saved CDW.pkl model
Uploaded model/CDW.pkl
Uploaded x_scaler/CDW.pkl
Uploaded y_scaler/CDW.pkl
Saved WFC.pkl model
Uploaded model/WFC.pkl
Uploaded x_scaler/WFC.pkl
Uploaded y_scaler/WFC.pkl
Saved PWR.pkl model
Uploaded model/PWR.pkl
Uploaded x_scaler/PWR.pkl
Uploaded y_scaler/PWR.pkl
Saved ZTS.pkl model
Uploaded model/ZTS.pkl
Uploaded x_scaler/ZTS.pkl
Uploaded y_scaler/ZTS.pkl
Saved BK.pkl model
Uploaded model/BK.pkl
Uploaded x_scaler/BK.pkl
Uploaded y_scaler/BK.pkl
Saved IQV.pkl model
Uploaded model/IQV.pkl
Uploaded x_scaler/IQV.pkl
Uploaded y_scaler/IQV.pkl
Saved BALY.pkl model
Uploaded model/BALY.pkl
Uploaded x_scaler/BALY.pkl
Uploaded y_scaler/BALY.pkl
Saved SCHW.pkl model
Uploaded model/SCHW.pkl
Uploaded x_scaler/SCHW.pkl
Uploaded y_scaler/SCHW.pkl
Saved WRB.pkl model
Uploaded model/WRB.pkl
Uploaded x_scaler/WRB.pkl
Uploaded y_scaler/WRB.pkl
Saved GE.pkl model
Uploaded model/GE.pkl
Uploaded x_scaler/GE.pkl
Uploaded y_scaler/GE.pkl
Saved TRMB.pkl model
Uploaded model/TRMB.pkl
Uploa

# Miscellaneous

## Delete all objects in bucket

In [ ]:
# def delete_from_s3():
#     # Set up constants for uploading to S3 bucket
#     with open(f'/content/drive/MyDrive/Colab Notebooks/env/aws_access_key.txt', 'r') as f:
#         aws_access_key_id = f.read()

#     with open(f'/content/drive/MyDrive/Colab Notebooks/env/aws_secret_key.txt', 'r') as f:
#         aws_secret_access_key = f.read()

#     target_bucket = "fourquant-robotrader-models"

#     # Set up S3 client
#     s3 = boto3.client('s3',
#                     aws_access_key_id = aws_access_key_id,
#                     aws_secret_access_key = aws_secret_access_key)

#     response = s3.list_objects_v2(Bucket=target_bucket)
#     for object in response['Contents']:
#         s3.delete_object(Bucket=target_bucket, Key=object['Key'])
#         print(f'Deleted {object}')

#     return

In [ ]:
# delete_from_s3()